# Prepare qmaps for variant callers results postprocessing

In [3]:
import json
import pandas as pd
import os

In [4]:
samples = json.load( open( "../../../cases_ids.json", "rb" ) )
samples

{'case1': {'normal': 'AQ5175',
  'tumor1': 'AQ5181',
  'tumor2': 'AQ5187',
  'sex': 'female'},
 'case2': {'normal': 'AQ5176',
  'tumor1': 'AQ5182',
  'tumor2': 'AQ5188',
  'sex': 'male'},
 'case3': {'normal': 'AQ5174',
  'tumor1': 'AQ5180',
  'tumor2': 'AQ5186',
  'sex': 'female',
  'kidney': 'AX4954',
  'liver': 'AX4955',
  'pancreas': 'AX4956',
  'heart': 'AX4957',
  'clone1': 'AX4958',
  'clone2': 'AX4961',
  'mother': 'AW8063',
  'father': 'AW8064',
  'lung': 'AX4962',
  'medulla': 'AX4963',
  'spleen': 'AX4964',
  'brain': 'AX4965',
  'bma': 'AX4966'},
 'case4': {'normal': 'AW8061',
  'tumor1': 'AW8050',
  'tumor2': 'AW8051',
  'sex': 'female'}}

# Create folder tree for output


In [13]:
# create a list with all the necessary folders for all the processed files

path = './output/' #change accordingly to another directory if needed
folders = []
folders.append(path)
for pt in samples.keys():
    pt_folder = path+pt+'/'
    folders.append(pt_folder)
    for t in ['tumor1','tumor2']:
        tumor = samples[pt][t]
        normal = samples[pt]['normal']
        t_vs_n = tumor + '_vs_' + normal
        t_folder = pt_folder + t_vs_n + '/'
        vcf_folder = t_folder +'vcf_processing/'
        folders.append(t_folder)
        folders.append(vcf_folder)
        for caller in ['mutect','strelka','sage','intersect']:
            caller_folder = vcf_folder + caller
            folders.append(caller_folder)
        vep_folder = t_folder +'process_vep_output/'
        vep2_folder = t_folder +'vep_processing'
        cnv_folder = t_folder +'process_cnv/'
        purple_folder = cnv_folder +'purple/'
        sv_folder = t_folder +'process_sv/'
        gridds_folder = sv_folder +'gridds/'
        folders.append(vep_folder)
        folders.append(vep2_folder)
        folders.append(cnv_folder)
        folders.append(purple_folder)
        folders.append(sv_folder)
        folders.append(gridds_folder)
        filter_folder = t_folder +'filter_and_annot/'
        folders.append(filter_folder)
    n_folder = pt_folder+samples[pt]['normal']+'/'
    folders.append(n_folder)
    for step in ['vcf_processing/','vep_processing/', 'process_vep_output/','filter_and_annot/']:
        step_folder = n_folder + step
        haplo_folder = step_folder+'haplotype_caller/'
        folders.append(step_folder)
        folders.append(haplo_folder)
    sv_folder = n_folder+'process_sv/'
    gripss_folder = sv_folder+'gripss/'
    t1_folder = gripss_folder+'t1/'
    t2_folder = gripss_folder+'t2/'
    folders.append(sv_folder)
    folders.append(gripss_folder)
    folders.append(t1_folder)
    folders.append(t2_folder)
folders

['./output/',
 './output/case1/',
 './output/case1/AQ5181_vs_AQ5175/',
 './output/case1/AQ5181_vs_AQ5175/vcf_processing/',
 './output/case1/AQ5181_vs_AQ5175/vcf_processing/mutect',
 './output/case1/AQ5181_vs_AQ5175/vcf_processing/strelka',
 './output/case1/AQ5181_vs_AQ5175/vcf_processing/sage',
 './output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect',
 './output/case1/AQ5181_vs_AQ5175/process_vep_output/',
 './output/case1/AQ5181_vs_AQ5175/vep_processing',
 './output/case1/AQ5181_vs_AQ5175/process_cnv/',
 './output/case1/AQ5181_vs_AQ5175/process_cnv/purple/',
 './output/case1/AQ5181_vs_AQ5175/process_sv/',
 './output/case1/AQ5181_vs_AQ5175/process_sv/gridds/',
 './output/case1/AQ5181_vs_AQ5175/filter_and_annot/',
 './output/case1/AQ5187_vs_AQ5175/',
 './output/case1/AQ5187_vs_AQ5175/vcf_processing/',
 './output/case1/AQ5187_vs_AQ5175/vcf_processing/mutect',
 './output/case1/AQ5187_vs_AQ5175/vcf_processing/strelka',
 './output/case1/AQ5187_vs_AQ5175/vcf_processing/sage',
 './output/c

In [14]:
#create folders
for folder in folders:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Somatic processing

## Process Mutect and SAGE vcf files

In [24]:
root_in_hmf = '/path/to/hmf_pipeline/output/'
root_in_sarek = '/path/to/sarek/output/'
root_out = './output/'

In [20]:
#commands for process vcfs from mutect and sage

python_file = './python_scripts/process_mutect_vcf.py'

commands = []

#SAGE
for pt in samples.keys():
    tumor1 = samples[pt]['tumor1']
    tumor2 = samples[pt]['tumor2']
    normal = samples[pt]['normal']
    sample = tumor1 + '_vs_' + normal
    in_file = os.path.join(root_in_hmf,'sage_somatic',tumor1+'.sage.somatic.filtered.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','sage'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor1 + ' -n_id ' + normal
    commands.append(command)
    sample = tumor2 + '_vs_' + normal
    in_file = os.path.join(root_in_hmf,'sage_somatic',tumor2+'.sage.somatic.filtered.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','sage'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor2 + ' -n_id ' + normal
    commands.append(command)
    
#Mutect
for pt in samples.keys():
    tumor1 = samples[pt]['tumor1']
    tumor2 = samples[pt]['tumor2']
    normal = samples[pt]['normal']
    sample = tumor1 + '_vs_' + normal
    in_file = os.path.join(root_in_sarek,'variant_calling','mutect2',sample,sample+'.mutect2.filtered.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','mutect'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor1 + ' -n_id ' + normal
    commands.append(command)
    tumor = samples[pt]['tumor2']
    sample = tumor2 + '_vs_' + normal
    in_file = os.path.join(root_in_sarek,'variant_calling','mutect2',sample,sample+'.mutect2.filtered.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','mutect'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor2 + ' -n_id ' + normal
    commands.append(command)
commands

['python ./python_scripts/process_mutect_vcf.py -i /path/to/hmf_pipeline/output/sage_somatic/AQ5181.sage.somatic.filtered.vcf.gz -o ./output/case1/AQ5181_vs_AQ5175/vcf_processing/sage/ -t_id AQ5181 -n_id AQ5175',
 'python ./python_scripts/process_mutect_vcf.py -i /path/to/hmf_pipeline/output/sage_somatic/AQ5187.sage.somatic.filtered.vcf.gz -o ./output/case1/AQ5187_vs_AQ5175/vcf_processing/sage/ -t_id AQ5187 -n_id AQ5175',
 'python ./python_scripts/process_mutect_vcf.py -i /path/to/hmf_pipeline/output/sage_somatic/AQ5182.sage.somatic.filtered.vcf.gz -o ./output/case2/AQ5182_vs_AQ5176/vcf_processing/sage/ -t_id AQ5182 -n_id AQ5176',
 'python ./python_scripts/process_mutect_vcf.py -i /path/to/hmf_pipeline/output/sage_somatic/AQ5188.sage.somatic.filtered.vcf.gz -o ./output/case2/AQ5188_vs_AQ5176/vcf_processing/sage/ -t_id AQ5188 -n_id AQ5176',
 'python ./python_scripts/process_mutect_vcf.py -i /path/to/hmf_pipeline/output/sage_somatic/AQ5180.sage.somatic.filtered.vcf.gz -o ./output/case3/A

In [21]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 20G','[jobs]']
qmap_file = qmap_pre_params + commands

In [23]:
#Save qmap file

with open('./qmap_files/01_mutect_sage_process.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Process strelka vcf

In [26]:
#commands for process vcfs from strelka

python_file = './python_scripts/process_strelka_v2.9.10_vcf.py'

commands = []

#Strelka
for pt in samples.keys():
    tumor1 = samples[pt]['tumor1']
    tumor2 = samples[pt]['tumor2']
    normal = samples[pt]['normal']
    sample = tumor1 + '_vs_' + normal
    in_file = os.path.join(root_in_sarek,'variant_calling','strelka',sample,sample+'.strelka.somatic_snvs.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','strelka'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor1 + ' -n_id ' + normal
    commands.append(command)
    tumor = samples[pt]['tumor2']
    sample = tumor2 + '_vs_' + normal
    in_file = os.path.join(root_in_sarek,'variant_calling','strelka',sample,sample+'.strelka.somatic_snvs.vcf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','strelka'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -t_id ' + tumor2 + ' -n_id ' + normal
    commands.append(command)
commands

['python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5181_vs_AQ5175/AQ5181_vs_AQ5175.strelka.somatic_snvs.vcf.gz -o ./output/case1/AQ5181_vs_AQ5175/vcf_processing/strelka/ -t_id AQ5181 -n_id AQ5175',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5187_vs_AQ5175/AQ5187_vs_AQ5175.strelka.somatic_snvs.vcf.gz -o ./output/case1/AQ5187_vs_AQ5175/vcf_processing/strelka/ -t_id AQ5187 -n_id AQ5175',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5182_vs_AQ5176/AQ5182_vs_AQ5176.strelka.somatic_snvs.vcf.gz -o ./output/case2/AQ5182_vs_AQ5176/vcf_processing/strelka/ -t_id AQ5182 -n_id AQ5176',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5188_vs_AQ5176/AQ5188_vs_AQ5176.strelka.somatic_snvs.vcf.gz -o ./output/case2/AQ5188_vs_AQ5176/vcf_processing/strelka/ -t_id AQ

In [27]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 20G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 20G',
 '[jobs]',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5181_vs_AQ5175/AQ5181_vs_AQ5175.strelka.somatic_snvs.vcf.gz -o ./output/case1/AQ5181_vs_AQ5175/vcf_processing/strelka/ -t_id AQ5181 -n_id AQ5175',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5187_vs_AQ5175/AQ5187_vs_AQ5175.strelka.somatic_snvs.vcf.gz -o ./output/case1/AQ5187_vs_AQ5175/vcf_processing/strelka/ -t_id AQ5187 -n_id AQ5175',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/variant_calling/strelka/AQ5182_vs_AQ5176/AQ5182_vs_AQ5176.strelka.somatic_snvs.vcf.gz -o ./output/case2/AQ5182_vs_AQ5176/vcf_processing/strelka/ -t_id AQ5182 -n_id AQ5176',
 'python ./python_scripts/process_strelka_v2.9.10_vcf.py -i /path/to/sarek/output/vari

In [28]:
#Save qmap file

with open('./qmap_files/02_strelka_process.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Intersect 3 callers and output by chromosome

In [30]:
#commands for process vcfs from HMF

python_file = './python_scripts/intersect_callers.py'

commands = []

for pt in samples.keys():
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    sample = tumor + '_vs_' + normal
    in_file_sage = os.path.join(root_out,pt,sample,'vcf_processing','sage',sample+'_process.maf.gz')
    in_file_mutect = os.path.join(root_out,pt,sample,'vcf_processing','mutect',sample+'_process.maf.gz')
    in_file_strelka = os.path.join(root_out,pt,sample,'vcf_processing','strelka',sample+'_process.maf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','intersect'+'/')
    command = 'python ' + python_file + ' -sa ' + in_file_sage + ' -mu ' + in_file_mutect + ' -st ' + in_file_strelka + ' -o ' + out_dir + ' -sn ' + sample + ' -c '
    commands.append(command)
    tumor = samples[pt]['tumor2']
    sample = tumor + '_vs_' + normal
    in_file_sage = os.path.join(root_out,pt,sample,'vcf_processing','sage',sample+'_process.maf.gz')
    in_file_mutect = os.path.join(root_out,pt,sample,'vcf_processing','mutect',sample+'_process.maf.gz')
    in_file_strelka = os.path.join(root_out,pt,sample,'vcf_processing','strelka',sample+'_process.maf.gz')
    out_dir = os.path.join(root_out,pt,sample,'vcf_processing','intersect'+'/')
    command = 'python ' + python_file + ' -sa ' + in_file_sage + ' -mu ' + in_file_mutect + ' -st ' + in_file_strelka + ' -o ' + out_dir + ' -sn ' + sample + ' -c '
    commands.append(command)
    
commands

['python ./python_scripts/intersect_callers.py -sa ./output/case1/AQ5181_vs_AQ5175/vcf_processing/sage/AQ5181_vs_AQ5175_process.maf.gz -mu ./output/case1/AQ5181_vs_AQ5175/vcf_processing/mutect/AQ5181_vs_AQ5175_process.maf.gz -st ./output/case1/AQ5181_vs_AQ5175/vcf_processing/strelka/AQ5181_vs_AQ5175_process.maf.gz -o ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/ -sn AQ5181_vs_AQ5175 -c ',
 'python ./python_scripts/intersect_callers.py -sa ./output/case1/AQ5187_vs_AQ5175/vcf_processing/sage/AQ5187_vs_AQ5175_process.maf.gz -mu ./output/case1/AQ5187_vs_AQ5175/vcf_processing/mutect/AQ5187_vs_AQ5175_process.maf.gz -st ./output/case1/AQ5187_vs_AQ5175/vcf_processing/strelka/AQ5187_vs_AQ5175_process.maf.gz -o ./output/case1/AQ5187_vs_AQ5175/vcf_processing/intersect/ -sn AQ5187_vs_AQ5175 -c ',
 'python ./python_scripts/intersect_callers.py -sa ./output/case2/AQ5182_vs_AQ5176/vcf_processing/sage/AQ5182_vs_AQ5176_process.maf.gz -mu ./output/case2/AQ5182_vs_AQ5176/vcf_processing/mutect

In [31]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 20G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 20G',
 '[jobs]',
 'python ./python_scripts/intersect_callers.py -sa ./output/case1/AQ5181_vs_AQ5175/vcf_processing/sage/AQ5181_vs_AQ5175_process.maf.gz -mu ./output/case1/AQ5181_vs_AQ5175/vcf_processing/mutect/AQ5181_vs_AQ5175_process.maf.gz -st ./output/case1/AQ5181_vs_AQ5175/vcf_processing/strelka/AQ5181_vs_AQ5175_process.maf.gz -o ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/ -sn AQ5181_vs_AQ5175 -c ',
 'python ./python_scripts/intersect_callers.py -sa ./output/case1/AQ5187_vs_AQ5175/vcf_processing/sage/AQ5187_vs_AQ5175_process.maf.gz -mu ./output/case1/AQ5187_vs_AQ5175/vcf_processing/mutect/AQ5187_vs_AQ5175_process.maf.gz -st ./output/case1/AQ5187_vs_AQ5175/vcf_processing/strelka/AQ5187_vs_AQ5175_process.maf.gz -o ./output/case1/AQ5187_vs_AQ5175/vcf_processing/intersect/ -sn AQ5187_vs_AQ5175 -c ',
 'python ./python_scripts/intersect_callers

In [32]:
#Save qmap file

with open('./qmap_files/03_intersect.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Run VEP 101 from intersect

In [35]:
gnomad_url = '/path/to/gnomad/data/v3.0.0/hg38/'
#run vep command

chroms = list(range(1,23))
chroms = ['chr'+str(chrom) for chrom in chroms]
sex_chroms = ['chrX','chrY']
chroms = chroms + sex_chroms

vep_commands = []
for pt in samples.keys():
    tumor1 = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    sample = tumor1 + '_vs_' + normal
    for chrom in chroms:
        pt_url = os.path.join(root_out,pt,sample,'vcf_processing','intersect',sample + '_' + chrom + '.maf.gz')
        chr_file = 'gnomad.genomes.r3.0.sites.'+chrom+'.vcf.bgz'
        gnomad_chr_file = os.path.join(gnomad_url,chr_file)
        out_file = pt_url.replace('vcf_processing/intersect/', 'vep_processing/')
        out_file = out_file.replace('.maf.gz', '_vep.txt')
        vep_command = 'vep -i '+ pt_url + ' -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /path/to/vep  --custom ' + gnomad_chr_file + ',gnomADg,vcf,exact,0,AF,NFE > ' + out_file
        vep_commands.append(vep_command)
    tumor2 = samples[pt]['tumor2']
    sample = tumor2 + '_vs_' + normal
    for chrom in chroms:
        pt_url = os.path.join(root_out,pt,sample,'vcf_processing','intersect',sample + '_' + chrom + '.maf.gz')
        chr_file = 'gnomad.genomes.r3.0.sites.'+chrom+'.vcf.bgz'
        gnomad_chr_file = os.path.join(gnomad_url,chr_file)
        out_file = pt_url.replace('vcf_processing/intersect/', 'vep_processing/')
        out_file = out_file.replace('.maf.gz', '_vep.txt')
        vep_command = 'vep -i '+ pt_url + ' -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /path/to/vep  --custom ' + gnomad_chr_file + ',gnomADg,vcf,exact,0,AF,NFE > ' + out_file
        vep_commands.append(vep_command)
        
vep_commands

['vep -i ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/AQ5181_vs_AQ5175_chr1.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /path/to/vep  --custom /path/to/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr1.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE > ./output/case1/AQ5181_vs_AQ5175/vep_processing/AQ5181_vs_AQ5175_chr1_vep.txt',
 'vep -i ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/AQ5181_vs_AQ5175_chr2.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /path/to/vep  --custom /path/to/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr2.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE > ./output/case1/AQ5181_vs_AQ5175/vep_processing/AQ5181_vs_AQ5175_chr2_vep.txt',
 'vep -i ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/AQ5181_vs_AQ5175_chr3.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline 

In [36]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate vep101','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + vep_commands
qmap_file
with open('./qmap_files/04_vep_gnomad_v3.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## process_vep101.py

In [39]:
#run process_vep command

python_script = './python_scripts/process_vep101.py'
process_vep_commands = []
for pt in samples.keys():
    tumor1 = samples[pt]['tumor1']
    tumor2 = samples[pt]['tumor2']
    normal = samples[pt]['normal']
    t1_vs_n = tumor1 + '_vs_' + normal
    t2_vs_n = tumor2 + '_vs_' + normal
    maf_url = os.path.join(root_out,pt,t1_vs_n,'vcf_processing','intersect/')
    vep_url = os.path.join(root_out,pt,t1_vs_n,'vep_processing/')
    command = 'python '+python_script+' --path_input_vep '+vep_url+' --path_input_maf '+maf_url+' --file_name '+t1_vs_n +' --cores 8'
    process_vep_commands.append(command)
    maf_url = os.path.join(root_out,pt,t2_vs_n,'vcf_processing','intersect/')
    vep_url = os.path.join(root_out,pt,t2_vs_n,'vep_processing/')
    command = 'python '+python_script+' --path_input_vep '+vep_url+' --path_input_maf '+maf_url+' --file_name '+t2_vs_n +' --cores 8'
    process_vep_commands.append(command)
    
    
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 8','memory = 15G','[jobs]']
qmap_file = qmap_pre_params + process_vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 8',
 'memory = 15G',
 '[jobs]',
 'python ./python_scripts/process_vep101.py --path_input_vep ./output/case1/AQ5181_vs_AQ5175/vep_processing/ --path_input_maf ./output/case1/AQ5181_vs_AQ5175/vcf_processing/intersect/ --file_name AQ5181_vs_AQ5175 --cores 8',
 'python ./python_scripts/process_vep101.py --path_input_vep ./output/case1/AQ5187_vs_AQ5175/vep_processing/ --path_input_maf ./output/case1/AQ5187_vs_AQ5175/vcf_processing/intersect/ --file_name AQ5187_vs_AQ5175 --cores 8',
 'python ./python_scripts/process_vep101.py --path_input_vep ./output/case2/AQ5182_vs_AQ5176/vep_processing/ --path_input_maf ./output/case2/AQ5182_vs_AQ5176/vcf_processing/intersect/ --file_name AQ5182_vs_AQ5176 --cores 8',
 'python ./python_scripts/process_vep101.py --path_input_vep ./output/case2/AQ5188_vs_AQ5176/vep_processing/ --path_input_maf ./output/case2/AQ5188_vs_AQ5176/vcf_processing/int

In [40]:
with open('./qmap_files/05_process_vep.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

Before proceding, we need to calculate the CCF and establish the threshold for clonality.\
This is performed in this notebook: ```TMB_and_CCF_analysis.ipynb```

##  filter_and_annot.py

In [7]:
#run process_vep command
platinum_results_url = '/workspace/datasets/sjd_seq/platinum_results/20220809/'
mafs_url = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'
ccf_json_path = '/workspace/projects/sjd_pediatric_tumors/code/ccf_thresholds2.json'
ccf_dict = json.load(open(ccf_json_path,'r'))
process_vep_commands = []
for pt in pts:
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    t_vs_n = tumor + '_vs_' + normal
    #tumor1
    sample = pt + '_t1'
    ccf = ccf_dict[sample]
    vep_url = os.path.join(mafs_url,pt,t_vs_n,'process_vep_output/')
    cnv_url = platinum_results_url + pt +'-t1-allsamples-t1/purple/'+tumor+'.purple.cnv.somatic.tsv'
    purity_url = platinum_results_url + pt +'-t1-allsamples-t1/purple/'+tumor+'.purple.purity.tsv'
    output_url = os.path.join(mafs_url,pt,t_vs_n,'filter_and_annot/')
    script =  '/workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py'
    command = 'python '+script+' -i '+vep_url+' -o '+output_url+' -t_id '+tumor + ' -n_id ' +normal + ' -ccf '+str(ccf) + ' -c intersect' + ' -cnv ' +cnv_url + ' -pur '+purity_url
    process_vep_commands.append(command)
    #tumor2
    tumor = samples[pt]['tumor2']
    t_vs_n = tumor + '_vs_' + normal
    sample = pt + '_t2'
    ccf = ccf_dict[sample]
    vep_url = os.path.join(mafs_url,pt,t_vs_n,'process_vep_output/')
    cnv_url = platinum_results_url + pt +'-t2-allsamples-t2/purple/'+tumor+'.purple.cnv.somatic.tsv'
    purity_url = platinum_results_url + pt +'-t2-allsamples-t2/purple/'+tumor+'.purple.purity.tsv'
    output_url = os.path.join(mafs_url,pt,t_vs_n,'filter_and_annot/')
    command = 'python '+script+' -i '+vep_url+' -o '+output_url+' -t_id '+tumor + ' -n_id ' +normal + ' -ccf '+str(ccf) + ' -c intersect'+ ' -cnv ' +cnv_url + ' -pur '+purity_url
    process_vep_commands.append(command)
    
    
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 8','memory = 15G','[jobs]']
qmap_file = qmap_pre_params + process_vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 8',
 'memory = 15G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_vep_output/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/filter_and_annot/ -t_id AQ5180 -n_id AQ5174 -ccf 0.4795411737486141 -c intersect -cnv /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.cnv.somatic.tsv -pur /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.purity.tsv',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_vep_output/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/2

In [8]:
with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_filter_and_annot_platinum.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

##  filter_and_annot.py (with another threshold for pt7)

In [3]:
#run process_vep command
platinum_results_url = '/workspace/datasets/sjd_seq/platinum_results/20220809/'
mafs_url = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'
ccf_json_path = '/workspace/projects/sjd_pediatric_tumors/code/ccf_thresholds2.json'
ccf_dict = json.load(open(ccf_json_path,'r'))
process_vep_commands = []
for pt in pts:
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    t_vs_n = tumor + '_vs_' + normal
    #tumor1
    sample = pt + '_t1'
    ccf = ccf_dict[sample]
    vep_url = os.path.join(mafs_url,pt,t_vs_n,'process_vep_output/')
    cnv_url = platinum_results_url + pt +'-t1-allsamples-t1/purple/'+tumor+'.purple.cnv.somatic.tsv'
    purity_url = platinum_results_url + pt +'-t1-allsamples-t1/purple/'+tumor+'.purple.purity.tsv'
    output_url = os.path.join(mafs_url,pt,t_vs_n,'filter_and_annot/')
    script =  '/workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py'
    command = 'python '+script+' -i '+vep_url+' -o '+output_url+' -t_id '+tumor + ' -n_id ' +normal + ' -ccf '+str(ccf) + ' -c intersect' + ' -cnv ' +cnv_url + ' -pur '+purity_url
    process_vep_commands.append(command)
    #tumor2
    tumor = samples[pt]['tumor2']
    t_vs_n = tumor + '_vs_' + normal
    sample = pt + '_t2'
    ccf = ccf_dict[sample]
    vep_url = os.path.join(mafs_url,pt,t_vs_n,'process_vep_output/')
    cnv_url = platinum_results_url + pt +'-t2-allsamples-t2/purple/'+tumor+'.purple.cnv.somatic.tsv'
    purity_url = platinum_results_url + pt +'-t2-allsamples-t2/purple/'+tumor+'.purple.purity.tsv'
    output_url = os.path.join(mafs_url,pt,t_vs_n,'filter_and_annot/')
    command = 'python '+script+' -i '+vep_url+' -o '+output_url+' -t_id '+tumor + ' -n_id ' +normal + ' -ccf '+str(ccf) + ' -c intersect'+ ' -cnv ' +cnv_url + ' -pur '+purity_url
    process_vep_commands.append(command)
    
    
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 8','memory = 15G','[jobs]']
qmap_file = qmap_pre_params + process_vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 8',
 'memory = 15G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_vep_output/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/filter_and_annot/ -t_id AQ5180 -n_id AQ5174 -ccf 0.8682485982482773 -c intersect -cnv /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.cnv.somatic.tsv -pur /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.purity.tsv',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_vep_output/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/2

In [4]:
with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230721_filter_and_annot_platinum.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Process vcf from GRIDDS (SV)

In [77]:
python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py'
genomic_positions = '/workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz'
canonical_transcripts = '/workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv'

commands = []
for pt in pts:
    #tumor1
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    input_vcf = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t1-allsamples-t1/purple/'+tumor+'.purple.sv.vcf.gz'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+tumor+'_vs_'+normal+'/process_sv/gridds/'
    command = 'python '+python_file+' -i '+input_vcf+' -o '+output_dir+' -gp '+genomic_positions+' -ct '+canonical_transcripts+' -t_id '+tumor+' -n_id '+normal
    commands.append(command)
    #tumor2
    tumor = samples[pt]['tumor2']
    input_vcf = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t2-allsamples-t2/purple/'+tumor+'.purple.sv.vcf.gz'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+tumor+'_vs_'+normal+'/process_sv/gridds/'
    command = 'python '+python_file+' -i '+input_vcf+' -o '+output_dir+' -gp '+genomic_positions+' -ct '+canonical_transcripts+' -t_id '+tumor+' -n_id '+normal
    commands.append(command)
commands

['python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.sv.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_sv/gridds/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv -t_id AQ5180 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/purple/AQ5186.purple.sv.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_sv/gridds/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv -t_id AQ5186 -n_id AQ5174'

In [78]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.sv.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_sv/gridds/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv -t_id AQ5180 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/purple/AQ5186.purple.sv.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_sv/gridds/ -gp /workspace/users/msanchezg/data/genomic_positions_

In [79]:
#Save qmap file

with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_process_gridds_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Process cnv vcf from purple

In [80]:
python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py'

commands = []
for pt in pts:
    #tumor1
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    input_tsv = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t1-allsamples-t1/purple/'+tumor+'.purple.cnv.gene.tsv'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+tumor+'_vs_'+normal+'/process_cnv/purple/'
    command = 'python '+python_file+' -i '+input_tsv+' -o '+output_dir+' -t_id '+tumor+' -n_id '+normal
    commands.append(command)
    #tumor2
    tumor = samples[pt]['tumor2']
    input_tsv = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t2-allsamples-t2/purple/'+tumor+'.purple.cnv.gene.tsv'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+tumor+'_vs_'+normal+'/process_cnv/purple/'
    command = 'python '+python_file+' -i '+input_tsv+' -o '+output_dir+' -t_id '+tumor+' -n_id '+normal
    commands.append(command)
commands

['python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.cnv.gene.tsv -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_cnv/purple/ -t_id AQ5180 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/purple/AQ5186.purple.cnv.gene.tsv -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_cnv/purple/ -t_id AQ5186 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt2-t1-allsamples-t1/purple/AQ5181.purple.cnv.gene.tsv -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5181_vs_AQ5175/process_cnv/purple/ -t_id AQ5181 -n_id AQ5175',
 

In [81]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/purple/AQ5180.purple.cnv.gene.tsv -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5180_vs_AQ5174/process_cnv/purple/ -t_id AQ5180 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/purple/AQ5186.purple.cnv.gene.tsv -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5186_vs_AQ5174/process_cnv/purple/ -t_id AQ5186 -n_id AQ5174',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_cnv_purple.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt2-t1-allsamples-t1/purple/AQ5181.purple.cn

In [82]:
#Save qmap file

with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_process_cnv_purple_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

# Germline processing

## HaplotypeCaller vcf processing

In [83]:
#commands for process vcfs from HMF

input_sage = '/workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/AQ5174/germline_caller/AQ5174.germline.vcf.gz'
output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/'

root_in = '/workspace/datasets/sjd_seq/platinum_results/20220809/'
root_out = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'

python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py'

commands = []

#SAGE
for pt in pts:
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    sample = tumor + '_vs_' + normal
    in_file = os.path.join(root_in,pt+'-t1-allsamples-t1',normal,'germline_caller',normal+'.germline.vcf.gz')
    out_dir = os.path.join(root_out,pt,normal,'vcf_processing','haplotype_caller'+'/')
    command = 'python ' + python_file + ' -i ' + in_file + ' -o ' + out_dir + ' -n_id ' + normal + ' -c 24'
    commands.append(command)
    
commands

['python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/AQ5174/germline_caller/AQ5174.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/ -n_id AQ5174 -c 24',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt2-t1-allsamples-t1/AQ5175/germline_caller/AQ5175.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/vcf_processing/haplotype_caller/ -n_id AQ5175 -c 24',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt3-t1-allsamples-t1/AQ5176/germline_caller/AQ5176.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt3/AQ5176/vcf_processin

In [84]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 24','memory = 100G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 24',
 'memory = 100G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/AQ5174/germline_caller/AQ5174.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/ -n_id AQ5174 -c 24',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt2-t1-allsamples-t1/AQ5175/germline_caller/AQ5175.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/vcf_processing/haplotype_caller/ -n_id AQ5175 -c 24',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_haplotype_caller.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt3-t1-

In [85]:
#Save qmap file

with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_haplocall_germline_process_platinum_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Run vep on HaplotypeCaller

In [63]:
chroms = list(range(1,23))
chroms = ['chr'+str(chrom) for chrom in chroms]
sex_chroms = ['chrX','chrY']
chroms = chroms + sex_chroms

sample_url = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/AQ5174_chr1.maf'
mafs_url = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'

chr1_gnomad_url = '/workspace/datasets/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr1.vcf.bgz'
gnomad_url = '/workspace/datasets/gnomad/data/v3.0.0/hg38/'

In [64]:
#run vep command

vep_commands = []
for pt in pts:
    tumor1 = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    for chrom in chroms:
        pt_url = os.path.join(mafs_url,pt,normal,'vcf_processing','haplotype_caller',normal + '_' + chrom + '.maf.gz')
        chr_file = 'gnomad.genomes.r3.0.sites.'+chrom+'.vcf.bgz'
        gnomad_chr_file = os.path.join(gnomad_url,chr_file)
        out_file = pt_url.replace('vcf_processing/haplotype_caller/', 'vep_processing/haplotype_caller/')
        out_file = out_file.replace('.maf.gz', '_vep.txt')
        vep_command = 'vep -i '+ pt_url + ' -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom ' + gnomad_chr_file + ',gnomADg,vcf,exact,0,AF,NFE > ' + out_file
        vep_commands.append(vep_command)
        
vep_commands

['vep -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/AQ5174_chr1.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /workspace/datasets/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr1.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE > /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vep_processing/haplotype_caller/AQ5174_chr1_vep.txt',
 'vep -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/AQ5174_chr2.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /workspace/datasets/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr2.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE > /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vep_processing/hap

In [65]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate vep101','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate vep101',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'vep -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/AQ5174_chr1.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /workspace/datasets/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3.0.sites.chr1.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE > /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vep_processing/haplotype_caller/AQ5174_chr1_vep.txt',
 'vep -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/AQ5174_chr2.maf.gz -o STDOUT -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /workspace/datasets/gnomad/data/v3.0.0/hg38/gnomad.genomes.r3

In [66]:
with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_run_vep_gnomad_v3_hc_platinum_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Process vep output from HaplotypeCaller

In [67]:
chroms = list(range(1,23))
chroms = ['chr'+str(chrom) for chrom in chroms]
sex_chroms = ['chrX','chrY']
chroms = chroms + sex_chroms

#run process_vep command
mafs_url = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'
python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/process_vep101_haplocall.py'
process_vep_commands = []
for pt in pts:
    for chrom in chroms:
        normal = samples[pt]['normal']
        file_name = normal + '_' + chrom + '_vep.txt'
        maf_url = os.path.join(mafs_url,pt,normal,'vcf_processing','haplotype_caller/')
        vep_url = os.path.join(mafs_url,pt,normal,'vep_processing','haplotype_caller/')
        command = 'python '+python_file+' --path_input_vep '+vep_url+' --path_input_maf '+maf_url+' --file_name '+file_name +' -c 16'
        process_vep_commands.append(command)

qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 16','memory = 100G','[jobs]']
qmap_file = qmap_pre_params + process_vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 16',
 'memory = 100G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_vep101_haplocall.py --path_input_vep /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vep_processing/haplotype_caller/ --path_input_maf /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/ --file_name AQ5174_chr1_vep.txt -c 16',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_vep101_haplocall.py --path_input_vep /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vep_processing/haplotype_caller/ --path_input_maf /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/vcf_processing/haplotype_caller/ --file_name AQ5174_chr2_vep.txt -c 16',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_vep101_haplocall

In [68]:
with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230201_process_vep_haplocall_platinum.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Filter and prepare germline alterations table (HaplotypeCaller)

In [89]:
python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py'

commands = []
for pt in pts:
    #tumor1
    normal = samples[pt]['normal']
    input_dir =  '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+normal+'/process_vep_output/haplotype_caller/'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+normal+'/filter_and_annot/haplotype_caller/'
    command = 'python '+python_file+' -i '+input_dir+' -o '+output_dir+' -n_id '+normal+' -c hc --gnomad_threshold 0.01'
    commands.append(command)
commands

['python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_vep_output/haplotype_caller/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/filter_and_annot/haplotype_caller/ -n_id AQ5174 -c hc --gnomad_threshold 0.01',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/process_vep_output/haplotype_caller/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/filter_and_annot/haplotype_caller/ -n_id AQ5175 -c hc --gnomad_threshold 0.01',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt3/AQ5176/process_vep_output/haplotype_caller/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt3/AQ51

In [90]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_vep_output/haplotype_caller/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/filter_and_annot/haplotype_caller/ -n_id AQ5174 -c hc --gnomad_threshold 0.01',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/process_vep_output/haplotype_caller/ -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt2/AQ5175/filter_and_annot/haplotype_caller/ -n_id AQ5175 -c hc --gnomad_threshold 0.01',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/filter_and_annot_muts.py -i /workspace/projects/sjd_pediatric_tumor

In [91]:
#Save qmap file

with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230206_filter_and_annot_germline_hc_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

## Process vcf from GRIPSS (SV)

In [21]:
python_file = '/workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py'
genomic_positions = '/workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz'
canonical_transcripts = '/workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv'

commands = []
for pt in pts:
    #tumor1
    tumor = samples[pt]['tumor1']
    normal = samples[pt]['normal']
    input_vcf = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t1-allsamples-t1/gripss_germline/'+normal+'.gripss.filtered.germline.vcf.gz'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+normal+'/process_sv/gripss/t1/'
    command = 'python '+python_file+' -i '+input_vcf+' -o '+output_dir+' -gp '+genomic_positions+' -ct '+canonical_transcripts+' -n_id '+normal+' --is_germline'
    commands.append(command)
    #tumor2
    tumor = samples[pt]['tumor2']
    input_vcf = '/workspace/datasets/sjd_seq/platinum_results/20220809/'+pt+'-t2-allsamples-t2/gripss_germline/'+normal+'.gripss.filtered.germline.vcf.gz'
    output_dir = '/workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/'+pt+'/'+normal+'/process_sv/gripss/t2/'
    command = 'python '+python_file+' -i '+input_vcf+' -o '+output_dir+' -gp '+genomic_positions+' -ct '+canonical_transcripts+' -n_id '+normal+' --is_germline'
    commands.append(command)
commands

['python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/gripss_germline/AQ5174.gripss.filtered.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_sv/gripss/t1/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv -n_id AQ5174 --is_germline',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/gripss_germline/AQ5174.gripss.filtered.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_sv/gripss/t2/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcr

In [22]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t1-allsamples-t1/gripss_germline/AQ5174.gripss.filtered.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_sv/gripss/t1/ -gp /workspace/users/msanchezg/data/genomic_positions_ensembl.txt.gz -ct /workspace/users/msanchezg/notebooks/protein_degradation/table_files/ensembl_canonical_transcripts.tsv -n_id AQ5174 --is_germline',
 'python /workspace/projects/sjd_pediatric_tumors/code/filters/process_gridds.py -i /workspace/datasets/sjd_seq/platinum_results/20220809/pt1-t2-allsamples-t2/gripss_germline/AQ5174.gripss.filtered.germline.vcf.gz -o /workspace/projects/sjd_pediatric_tumors/mafs_platinum/20220809/pt1/AQ5174/process_sv/gripss/t2/ -gp /workspace/user

In [23]:
#Save qmap file

with open('/workspace/projects/sjd_pediatric_tumors/code/qmap_files/230504_process_gridpss_germline_pt1_11.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)